In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer


In [18]:
class SimpleQuadratic(Layer):

  def __init__(self, units=32, activation=None):
    super(SimpleQuadratic, self).__init__()
    self.units = units
    self.activation = tf.keras.activations.get(activation)

  def build(self, input_shape):
    a_init = tf.random_normal_initializer()
    b_init = tf.random_normal_initializer()
    c_init = tf.zeros_initializer()

    a_init_val = a_init(shape=(input_shape[-1], self.units), dtype='float32')
    b_init_val = b_init(shape=(input_shape[-1], self.units), dtype='float32')
    c_init_val = c_init(shape=(self.units), dtype='float32')

    self.a = tf.Variable(initial_value=a_init_val, trainable=True)
    self.b = tf.Variable(initial_value=b_init_val, trainable=True)
    self.c = tf.Variable(initial_value=c_init_val, trainable=True)
  def call(self, inputs):
    x_squared = tf.math.square(inputs)
    x_squared_times_a = tf.linalg.matmul(x_squared, self.a)
    x_times_b = tf.linalg.matmul(inputs, self.b)
    x2a_plus_xb_plus_c = x_squared_times_a + x_times_b + self.c
    return self.activation(x2a_plus_xb_plus_c)


In [16]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train.shape

(60000,)

In [19]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    SimpleQuadratic(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2713 - accuracy: 0.9193
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1310 - accuracy: 0.9610
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0993 - accuracy: 0.9696
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0816 - accuracy: 0.9746
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0793 - accuracy: 0.9752


[0.0793345496058464, 0.9751999974250793]